# Sharing with Kronikare 13 July 2018
## Agenda
- [Demo on CascadedFCN](#CascadedFCN) 30 mins
- [Dicussion and Q&A](#Dicussion) 15 mins
- [Sprint Planning](#Sprint-Planning) 15 mins

# CascadedFCN
Paper suggest a noval approach to segment liver and lesions from CT and MRI images. <br>
There are 3 main steps:
1. Preprocessing (ref to figure3)
 - HU windowing for CT and N4Boas correction for MRI
 - Histogram Equalization
 - Augmentation (mirror, crop, elastic deformation, addition of noise)
2. 2-cascaded FCN (ref to figure6)
 - First segments liver
 - Then segments the leisons
3. Post processing
 - 3D Conditional Random Field

![Preprocessing](Preprocessing_from_paper.png)

![CascadedFCN](CascadedFCN_from_paper.png)

## How we are doing it
Since our images are RGB, we are not going to do the HU windowing/N4Bias Correction nor Histogram Equalization. <br>
We are just using Augmentation by [Augmentor](http://augmentor.readthedocs.io/en/master/) because we find that the Off-the-shelf Augmentation by Tensorflow has some artifacts (Extrapolation of pixels which makes images look funny). We are employing:
- Rotation and automatic Zoom
- Zoom
- Flipping Vertically and Horizontally
- Shearing

After doing the augmentation, we are going feed the UNet with the masks and images <br>
Due to memory limitation, we are using the .flowfromdirectory method from the ImageDataGenerator Class which reads batches of images and masks pairs from disk into memory. However, the off-the-shelf onehotencoding has some memory problem too, so some time was spent to do the onehotencoding after reading in the images and masks per batch.<br>
Below is an overview on how we are implementing the paper for our own use case
![Howwearedoing](CascadedFCN.png)

# Dicussion

# Sprint Planning

# Citations
[1] [CascaedFCN](arXiv:1702.05970) <br>

In [1]:
import os
import numpy as np
import json
import pandas as pd
from scipy.misc import imread

import keras
from keras.models import Model
from keras.layers import Conv2D, UpSampling2D, MaxPooling2D, Dropout, Cropping2D, Input, merge
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras import backend as K
from keras.utils import to_categorical

from metrics import f1 as f1_score


def UNet(filters_dims, activation='relu', kernel_initializer='glorot_uniform', padding='same'):
    inputs = Input((480, 640, 3))
    new_inputs = inputs
    conv_layers = []
    # Encoding Phase
    for i in range(len(filters_dims) - 1):
        conv = Conv2D(filters_dims[i], 3, activation=activation, padding=padding,
                      kernel_initializer=kernel_initializer)(new_inputs)
        conv = Conv2D(filters_dims[i], 3, activation=activation, padding=padding,
                      kernel_initializer=kernel_initializer)(conv)
        conv_layers.append(conv)
        new_inputs = MaxPooling2D(pool_size=(2, 2))(conv)
        # op = BatchNormalization()(op)

    # middle phase
    conv = Conv2D(filters_dims[-1], 3, activation=activation, padding=padding,
                  kernel_initializer=kernel_initializer)(new_inputs)
    conv = Conv2D(filters_dims[-1], 3, activation=activation, padding=padding,
                  kernel_initializer=kernel_initializer)(conv)
    new_inputs = Dropout(0.5)(conv)

    filters_dims.reverse()
    conv_layers.reverse()

    # Decoding Phase
    for i in range(1, len(filters_dims)):
        up = Conv2D(filters_dims[i], 3, activation=activation, padding=padding,
                    kernel_initializer=kernel_initializer)(UpSampling2D(size=(2, 2))(new_inputs))
        concat = merge([conv_layers[i-1], up], mode='concat', concat_axis=3)
        conv = Conv2D(filters_dims[i], 3, activation=activation, padding=padding,
                      kernel_initializer=kernel_initializer)(concat)
        new_inputs = Conv2D(filters_dims[i], 3, activation=activation, padding=padding,
                            kernel_initializer=kernel_initializer)(conv)
    outputs = Conv2D(2, 1, activation='softmax', padding='same',
                     kernel_initializer='glorot_uniform')(new_inputs)

    model = Model(input=inputs, output=outputs, name='UNet')
    model.compile(optimizer=Adam(lr=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'mse', f1_score])
    return model

/home/renjie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'metrics'